## Arranging the data - Create an utility matrix

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from collections import Counter



### Creating a DataFrame

In [3]:

def create_DataFrame(data):

    def from_iterable_unique(iterables): # I edited the function chain.from_iterable
        for it in iterables.str.split(', '):
            c1=Counter(it)
            set_it= set(it)
            for _ in set_it:
                yield c1.popitem()

    # calculate the amount of different cities in each session
    lens = data['cityCodes'].str.split(', ').apply(set).map(len)

    # create new dataframe, repeating or chaining as appropriate
    res = pd.DataFrame({'userId': np.repeat(data['userId'], lens),
                        'sessionId': np.repeat(data['sessionId'], lens),
                        'unique_cityCodes': np.repeat(data['cityCodes'], lens),
                        'city_searches': from_iterable_unique(data['cityCodes'])})

    res['city'], res['searches'] = zip(*res.city_searches)
    return res




### Transforming the data



In [4]:
def transforming_data (res):
    utility_matrix= res.pivot_table(index='sessionId',columns = 'city', values='searches', aggfunc=np.sum, fill_value=0) #it also merges the sessionId
    return utility_matrix



### remove unknown sessions

In [5]:
def remove_unknown_sessions (utility_matrix , unknown_sessions_list=[] ):
    utility_matrix=utility_matrix.drop(unknown_sessions_list, axis=0, errors='ignore') # drop the sessions in the list if they exist
    return utility_matrix
